### data_collection

In [ ]:
import cv2
import os
import numpy as np
from datetime import datetime

class DataCollector:
    def __init__(self):
        self.emotions = ['feliz', 'enojado', 'triste', 'sorprendido', 'neutral']
        self.base_path = 'dataset'
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def create_directories(self):
        for emotion in self.emotions:
            path = os.path.join(self.base_path, emotion)
            os.makedirs(path, exist_ok=True)

    def preprocess_image(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_roi = gray[y:y+h, x:x+w]
            resized_face = cv2.resize(face_roi, (48, 48))
            return resized_face
        return None

    def collect_data(self):
        self.create_directories()
        cap = cv2.VideoCapture(0)

        for emotion in self.emotions:
            count = 0
            print(f"Preparado para capturar expresión: {emotion}")
            input("Presiona Enter cuando estés listo...")

            while count < 140:
                ret, frame = cap.read()
                if not ret:
                    continue

                processed_face = self.preprocess_image(frame)
                if processed_face is not None:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                    filename = f"{emotion}_{timestamp}.jpg"
                    filepath = os.path.join(self.base_path, emotion, filename)
                    cv2.imwrite(filepath, processed_face)
                    count += 1

                cv2.putText(frame, f"Capturando {emotion}: {count}/140",
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Captura', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    collector = DataCollector()
    collector.collect_data()

# Created/Modified files during execution:
# - dataset/feliz/*.jpg
# - dataset/enojado/*.jpg
# - dataset/triste/*.jpg
# - dataset/sorprendido/*.jpg
# - dataset/neutral/*.jpg

Preparado para capturar expresión: feliz


### train_model

In [ ]:
import numpy as np
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class EmotionModelTrainer:
    def __init__(self):
        self.emotions = ['feliz', 'enojado', 'triste', 'sorprendido', 'neutral']
        self.data_path = 'dataset'
        self.model_path = 'emotion_model.h5'

    def load_data(self):
        X = []
        y = []

        for idx, emotion in enumerate(self.emotions):
            path = os.path.join(self.data_path, emotion)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                X.append(img)
                y.append(idx)

        X = np.array(X) / 255.0  # Normalización
        X = X.reshape(-1, 48, 48, 1)
        y = to_categorical(y, len(self.emotions))

        return train_test_split(X, y, test_size=0.2, random_state=42)

    def create_model(self):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            MaxPooling2D(2, 2),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Flatten(),
            Dense(64, activation='relu'),
            Dropout(0.5),
            Dense(len(self.emotions), activation='softmax')
        ])

        model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
        return model

    def train(self):
        X_train, X_test, y_train, y_test = self.load_data()
        model = self.create_model()

        history = model.fit(X_train, y_train,
                          validation_data=(X_test, y_test),
                          epochs=50,
                          batch_size=32)

        model.save(self.model_path)
        self.plot_training_history(history)

        # Evaluación final
        test_loss, test_acc = model.evaluate(X_test, y_test)
        print(f"\nPrecisión en el conjunto de prueba: {test_acc*100:.2f}%")

    def plot_training_history(self, history):
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Entrenamiento')
        plt.plot(history.history['val_accuracy'], label='Validación')
        plt.title('Precisión del Modelo')
        plt.xlabel('Época')
        plt.ylabel('Precisión')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Entrenamiento')
        plt.plot(history.history['val_loss'], label='Validación')
        plt.title('Pérdida del Modelo')
        plt.xlabel('Época')
        plt.ylabel('Pérdida')
        plt.legend()

        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.close()

if __name__ == "__main__":
    trainer = EmotionModelTrainer()
    trainer.train()

# Created/Modified files during execution:
# - emotion_model.h5
# - training_history.png

c:\Users\velez\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.1888 - loss: 1.6147 - val_accuracy: 0.2571 - val_loss: 1.6090
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2054 - loss: 1.6066 - val_accuracy: 0.2000 - val_loss: 1.6078
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2454 - loss: 1.5975 - val_accuracy: 0.1857 - val_loss: 1.5800
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2830 - loss: 1.5707 - val_accuracy: 0.5786 - val_loss: 1.4876
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3546 - loss: 1.4695 - val_accuracy: 0.4857 - val_loss: 1.3070
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5076 - loss: 1.3224 - val_accuracy: 0.6286 - val_loss: 1.0506
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5916 - loss: 1.1428 - val_accuracy: 0.6929 - val_loss: 0.8859
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.5991 - loss: 1.0175 - val_accuracy: 0.8786 - v

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9868 - loss: 0.0500

Precisión en el conjunto de prueba: 99.29%


### real_time_detection

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

class EmotionDetector:
    def __init__(self):
        self.emotions = ['feliz', 'enojado', 'triste', 'sorprendido', 'neutral']
        self.model = load_model('emotion_model.h5')
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )

    def preprocess_face(self, face_roi):
        face_roi = cv2.resize(face_roi, (48, 48))
        face_roi = face_roi / 255.0
        face_roi = face_roi.reshape(1, 48, 48, 1)
        return face_roi

    def detect_emotion(self):
        cap = cv2.VideoCapture(0)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                face_roi = gray[y:y+h, x:x+w]
                processed_face = self.preprocess_face(face_roi)
                prediction = self.model.predict(processed_face)
                emotion_idx = np.argmax(prediction)
                emotion = self.emotions[emotion_idx]
                confidence = prediction[0][emotion_idx] * 100

                # Dibujar rectángulo y etiqueta
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                label = f"{emotion} ({confidence:.1f}%)"
                cv2.putText(frame, label, (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                           (0, 255, 0), 2)

            cv2.imshow('Detector de Emociones', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = EmotionDetector()
    detector.detect_emotion()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━